In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


1. prepare data
  * read img:
    * opencv read in an img 
      * data type: np.ndarray 128 \* 128 \* 3 
    * have label?
      * if so return X, Y else Y
      * Y is required to be torch.LongTensor(int64) else batch_loss.backward() throws excp
  * deine your own Dataset & DataLoader.
    * train_set, valid_set, test_set
      * dataset:
        * transformer:
          * ToPIL
          * RandomHorizontalFlip
          * RandomRotation
          * ToTensor ----> tensor.Float32
    * data loader: implement \_\_len\_\_ and \_\_getitem\_\_
      * iterable, for i,data in numerate ( dataLoader) :
        * i for index and data for a batch.
        * batch size: batch_size 
  * split img into 3 sets.
  * lock parameters for latter training 
2. define model
  * cnn
    * channel 3 -> 64 -> 128 -> 256 -> 512 -> 512
      * covd2d
      * relu
      * normalize
      * maxpooling
  * Flatten output of CNN torch.view(2,-1)
  * fc
    * linear  512 * 4 * 4  --> 1024
    * linear  1024 -> 512
    * linear  512 ->11 dim( total 11 classes
3. implement operations between training.
  * define super params
    * batch_size
    * lr
    * statistics about batch_loss, batch_acc
  * define approches to calculating accuracy, loss.

In [49]:
import os
import cv2
import numpy as np
import torch.nn
import torchvision
import torch.optim
import torchvision.transforms as transform
from torch.utils.data import Dataset, DataLoader
import time


In [25]:
def read_img(path, label=False):
    # label means it's training/validation or testing.
    files = sorted(os.listdir(path))
    All_imgs = np.zeros((len(files), 128, 128, 3), dtype=np.uint8)
    All_labels = np.zeros(len(files),dtype=np.uint8)
    for i, each_file in enumerate(files):
        img = cv2.imread(os.path.join(path, each_file))
        # gives matrix of 128 * 128 * 3 where each ele of the matrix is a 3-dim vector.
        All_imgs[i, :, :,: ] = cv2.resize(img, (128, 128))
        # right_side: 128*128*3
        if label:
            All_labels[i] = each_file.split("_")[0]
            # there will certainly be data type transition.
        print(f'{each_file} done, {i} of total {len(files)}')
    if label:
        return All_imgs, All_labels
    else:
        return All_imgs

In [ ]:
work_dir = './gdrive/My Drive/Colab Notebooks/food-11'
train_X, train_Y = read_img(os.path.join(work_dir, 'training'), True)
test_X = read_img(os.path.join(work_dir, 'testing'), False)
valid_X, valid_Y = read_img(os.path.join(work_dir, 'validation'), True)

train_size = len(train_Y)
valid_size = len(valid_Y)
test_size = len(test_X)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Streaming output truncated to the last 5000 lines.
1777.jpg done, 1777 of total 3347
1778.jpg done, 1778 of total 3347
1779.jpg done, 1779 of total 3347
1780.jpg done, 1780 of total 3347
1781.jpg done, 1781 of total 3347
1782.jpg done, 1782 of total 3347
1783.jpg done, 1783 of total 3347
1784.jpg done, 1784 of total 3347
1785.jpg done, 1785 of total 3347
1786.jpg done, 1786 of total 3347
1787.jpg done, 1787 of total 3347
1788.jpg done, 1788 of total 3347
1789.jpg done, 1789 of total 3347
1790.jpg done, 1790 of total 3347
1791.jpg done, 1791 of total 3347
1792.jpg done, 1792 of total 3347
1793.jpg done, 1793 of total 3347
1794.jpg done, 1794 of total 3347
1795.jpg done, 1795 of total 3347
1796.jpg done, 1796 of total 3347
1797.jpg done, 1797 of total 3347
1798.jpg done, 1798 of total 3347
1799.jpg done, 1799 of total 3347
1800.jpg done, 1800 of total 3347
1801.jpg done, 1801 of total 3347
1802.jpg done, 1802 of total 3347
1803.jpg done, 1803 of total 3347
1804.jpg done, 1804 of total 33

9866

In [29]:
class ImgDataset(Dataset):
    def __init__(self, row_X, row_Y=None):
        self.Y = torch.LongTensor(row_Y) if row_Y is not None else None
        self.transformer = transform.Compose([
            transform.ToPILImage(),
            transform.RandomHorizontalFlip(),
            transform.RandomRotation(15),
            transform.ToTensor()
        ]
        ) if row_Y is not None \
            else \
            transform.Compose(
                [
                    transform.ToPILImage(),
                    transform.ToTensor()
                ]
            )
        temp = [self.transformer(i) for i in row_X]
        self.X = torch.zeros(len(temp), 3, 128, 128, dtype=torch.float32)
        for i, pic in enumerate(temp):
            self.X[i, :, :, :] = pic

    def __len__(self):
        return self.X.size()[0]

    def __getitem__(self, index):
        return self.X[index], self.Y[index] if self.Y is not None else self.X[index]


train_dataset = ImgDataset(train_X, train_Y)
valid_dataset = ImgDataset(valid_X, valid_Y)
test_dataset = ImgDataset(test_X)

In [35]:
class Classify(torch.nn.Module):
    def __init__(self):
        # input
        super().__init__()
        self.CNN = torch.nn.Sequential(
            # input a img,batch_size * 3 * 128 * 128
            torch.nn.Conv2d(3, 64, 3, 1, 1),  # givs 64 * 128 * 128
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2, 0),  # givs 64 * 64 * 64

            torch.nn.Conv2d(64, 128, 3, 1, 1),  # gives 128 * 64 * 64
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2, 0),  # givs 128 * 32 * 32

            torch.nn.Conv2d(128, 256, 3, 1, 1),  # gives 256 * 32 * 32
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2, 0),  # gives 256 * 16 * 16

            torch.nn.Conv2d(256, 512, 3, 1, 1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2, 0),  # gives 512 * 8 * 8

            torch.nn.Conv2d(512, 512, 3, 1, 1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2, 0),  # this gives 512 * 4 * 4
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(512 * 4 * 4, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 11)
        )

    def forward(self, tensor):
        temp = self.CNN(tensor)
        temp = temp.view(temp.size()[0],-1)
        return self.fc(temp)

In [57]:
Juan = Classify().to(device)
loss = torch.nn.CrossEntropyLoss().to(device)
opt = torch.optim.Adam(Juan.parameters(), lr=0.001)
batch_size = 128  # 128
iter_num = 30

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [58]:
for epoch in range(iter_num):
    epoch_start_time = time.time()  
    train_acc = 0
    train_total = 0
    train_loss = 0.0
    for data in train_loader:
        opt.zero_grad()
        temp=data[0].to(device)
        train_pred = Juan(temp)
        temp_loss = loss(train_pred, data[1].to(device))
        temp_loss.backward()
        opt.step()
        train_acc += np.sum(torch.argmax(train_pred.cpu(), dim=1).data.numpy() == data[1].view(1,- 1).data.numpy())
        train_total += data[1].size()[0]
        train_loss += temp_loss
        # train_pred : n * 11
        # data[1]  n

    with torch.no_grad():
        val_loss = 0.0
        val_total = 0
        val_acc = 0
        for i, data in enumerate(valid_loader):
            valid_pred = Juan(data[0].to(device))
            val_loss+= loss(valid_pred, data[1].to(device)).item()
            val_acc += np.sum(torch.argmax(valid_pred.cpu(), dim=1).data.numpy() == data[1].view(1, -1).data.numpy())
            val_total += len(data[1])
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, iter_num, time.time()-epoch_start_time, \
             train_acc/train_dataset.__len__(), train_loss/train_dataset.__len__(), val_acc/valid_dataset.__len__(), val_loss/valid_dataset.__len__()))

[001/030] 19.76 sec(s) Train Acc: 0.229982 Loss: 0.018491 | Val Acc: 0.297668 loss: 0.015570
[002/030] 19.97 sec(s) Train Acc: 0.318366 Loss: 0.015159 | Val Acc: 0.341983 loss: 0.014620
[003/030] 19.89 sec(s) Train Acc: 0.383337 Loss: 0.014013 | Val Acc: 0.390962 loss: 0.013620
[004/030] 19.72 sec(s) Train Acc: 0.426617 Loss: 0.012944 | Val Acc: 0.432945 loss: 0.012992
[005/030] 19.82 sec(s) Train Acc: 0.475066 Loss: 0.011833 | Val Acc: 0.476093 loss: 0.012207
[006/030] 19.89 sec(s) Train Acc: 0.504054 Loss: 0.011155 | Val Acc: 0.493878 loss: 0.011717
[007/030] 19.86 sec(s) Train Acc: 0.536692 Loss: 0.010491 | Val Acc: 0.509621 loss: 0.010958
[008/030] 19.82 sec(s) Train Acc: 0.572471 Loss: 0.009645 | Val Acc: 0.513994 loss: 0.011472
[009/030] 19.85 sec(s) Train Acc: 0.594871 Loss: 0.009182 | Val Acc: 0.547813 loss: 0.010406
[010/030] 19.84 sec(s) Train Acc: 0.624265 Loss: 0.008547 | Val Acc: 0.543440 loss: 0.010562
[011/030] 19.87 sec(s) Train Acc: 0.636225 Loss: 0.008292 | Val Acc: 0